In [ ]:
from openai import OpenAI
import time
openai_api_key = "EMPTY" # "EMPTY" if you don't have the api-key flag, change to the your custom api key 
openai_api_base = "http://0.0.0.0:8963/v1" # change to your host and port
""" Notice that these 2 can be set as enviroment variables of OPENAI_API_KEY and OPENAI_BASE_URL"""

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/HDD/models/ai-tutor/tokenizers/Qwen/Qwen2.5-32B-Instruct")
client = OpenAI(api_key=openai_api_key,base_url=openai_api_base,)

SYSTEM_PROMPT = """Bạn là một gia sư Toán học với ngôn ngữ Tiếng Việt. CHỈ trả lời thông tin về môn toán học.
Nhiệm vụ của bạn là giải bài toán một cách chi tiết từng bước.

KHÔNG trả lời cho môn học khác và phản hồi lịch sự là bạn không biết câu hỏi đó.
HÃY LUÔN PHẢN HỒI HOÀN TOÀN BẰNG TIẾNG VIỆT
"""

math_questions =[
        'Một đội bóng đá thi đấu trong một sân vận động có sức chứa 55 000 khán giả. Với giá mỗi vé là 100 nghìn đồng, số khán giả trung bình là 27 000 người. Qua thăm dò dư luận, người ta thấy rằng mỗi khi giá vé giảm thêm 10 nghìn đồng, sẽ có thêm khoảng 3000 khán giả. Hỏi ban tổ chức nên đặt giá vé là bao nhiêu để doanh thu từ tiền bán vé là lớn nhất?',
        'Trong các câu sau, câu nào là mệnh đề?\nA. Đi ngủ đi!\nB. Trung Quốc là nước đông dân nhất thế giới.\nC. Bạn học trường nào?\nD. Không được làm việc riêng trong giờ học.',
        'Cho hàm số y = f(x) có đạo hàm f’(x) = x2(x + 1)(x – 2) .Tìm số điểm cực trị của hàm số đã cho ',
        'Trước kì thi học sinh giỏi cấp huyện, phòng giáo dục bằng động viên đội tuyển học sinh giỏi của huyện có 600 người mỗi người 250 nghìn đồng. Theo như kế hoạch kinh phí ban đầu thì mỗi người nhận sẽ có 500 nghìn đồng.  Ngân sách của huyện chi cho mỗi người tăng thêm bao nhiêu phần trăm so với kế hoạch ban đầu? Chọn ngẫu nhiên một học sinh của đội tuyển.  Xác suất để chọn được học sinh khối lớp chưa được nhận là bao nhiêu?\nA. $\\frac{{117}}{{320}}$\nB. $\\frac{{11}}{{320}}$\nC. $\\frac{{13}}{{32}}$\nD. $\\frac{{13}}{{320}}$',
        'Máy bay trực thăng ra lệnh hạ ngẫu nhiên. Tỉ lệ nguy hiểm khi hạ trên bãi cỏ là 0,05. Nếu nguy hiểm thì máy bay trực thăng thứ nhất hạ trên bãi cỏ thì xác suất thành công của thí nghiệm thứ hai là 0,8. Nếu thí nghiệm thứ nhất không thành công thì xác suất thành công của thí nghiệm thứ hai là 0,3. Tìm xác suất của biến cố: “A: Cả hai lần thí nghiệm đều thành công”\nA. 0,48\nB. 0,12\nC. 0,3\nD. 0,6'
]


for question in math_questions:
    full_output =""

    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': question}
    ]
    
    response = client.chat.completions.create(
        model="qwen2.5-72b",
        messages=messages,
        max_tokens=1024,
        temperature=0.1,
        top_p= 0.9,
        extra_body={
        "repetition_penalty": 1.05,
    },
        stream=True,
        stream_options= {'include_usage' : True}
    )
    start = time.perf_counter()
    for part in response:
        if len(part.choices)>0:
            if token := part.choices[0].delta.content or "":
                print(token, end='', flush=True)
    print()
